Read .nc files

In [8]:
import xarray as xr

# Open the NetCDF file
file_path = r"D:\IPMA\Countries\Spain_mask.nc"
ds = xr.open_dataset(file_path)

# Display the dataset's structure and metadata
print("Dataset Overview:\n")
print(ds)

# List all variables in the dataset
print("\nAvailable Variables:")
for var in ds.variables:
    print(f"- {var}: {ds[var].dims}, {ds[var].shape}")


Dataset Overview:

<xarray.Dataset> Size: 23kB
Dimensions:    (latitude: 43, longitude: 65)
Coordinates:
  * latitude   (latitude) float64 344B 34.5 35.25 36.0 36.75 ... 64.5 65.25 66.0
  * longitude  (longitude) float64 520B -12.0 -11.25 -10.5 ... 34.5 35.25 36.0
Data variables:
    mask       (latitude, longitude) int64 22kB ...
Attributes:
    description:                 Mask for Spain, 1=inside, 0=outside
    grid_resolution:             0.75 x 0.75 degrees
    study_area_latitude_range:   34.5 to 66.01
    study_area_longitude_range:  -12 to 36.01

Available Variables:
- mask: ('latitude', 'longitude'), (43, 65)
- latitude: ('latitude',), (43,)
- longitude: ('longitude',), (65,)


View nc file in table

In [12]:
import xarray as xr
import pandas as pd

# Open the NetCDF file
nc_file = r"D:\IPMA\Countries\Italy_mask.nc" 
ds = xr.open_dataset(nc_file)

# Convert all variables to a DataFrame
df = ds.to_dataframe().reset_index()

# Display the first few rows
print(df.head())

df


   latitude  longitude  mask
0      34.5     -12.00     0
1      34.5     -11.25     0
2      34.5     -10.50     0
3      34.5      -9.75     0
4      34.5      -9.00     0


,latitude,longitude,mask
0,34.5,-12.00,0
1,34.5,-11.25,0
2,34.5,-10.50,0
3,34.5,-9.75,0
4,34.5,-9.00,0
...,...,...,...
2790,66.0,33.00,0
2791,66.0,33.75,0
2792,66.0,34.50,0
2793,66.0,35.25,0


Read .mat files

In [ ]:
from scipy.io import loadmat

data = loadmat(r"E:\IPMA\Sources\Virgilio\FRP\MODIS_2001.mat", struct_as_record=False, squeeze_me=True)

dados = data["dados"]  # Extract variable

print("Type of 'dados':", type(dados))

if hasattr(dados, "_fieldnames"):  # Check if it's a struct
    print("Struct fields:", dados._fieldnames)
    for field in dados._fieldnames:
        print(f"Field '{field}':", getattr(dados, field))

View .mat on map

In [ ]:
import scipy.io
import folium

# Step 1: Load the .mat file
mat_data = scipy.io.loadmat(r"E:\IPMA\Sources\Virgilio\FRP\MODIS_2001.mat")  # Replace 'your_file.mat' with your file's name

# Step 2: Inspect the contents of the .mat file to find the data
print(mat_data.keys())  # This will list all variables in the .mat file

# Step 3: Extract latitude and longitude (adjust if your data keys are different)
# Assuming that 'latitudes' and 'longitudes' are the keys in your .mat file
latitudes = mat_data['lat'].flatten()  # Flatten in case the data is in an array form
longitudes = mat_data['lon'].flatten()

# Step 4: Create a map centered around the first set of coordinates (you can adjust this)
my_map = folium.Map(location=[latitudes[0], longitudes[0]], zoom_start=10)

# Step 5: Add markers for each coordinate pair
for lat, lon in zip(latitudes, longitudes):
    folium.Marker([lat, lon]).add_to(my_map)

# Step 6: Save the map to an HTML file
my_map.save("map.html")

# Step 7: Instructions to view the map
print("Map saved as 'map.html'. Open it in a browser to view the map.")


View .csv file on table

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

file_path = r"E:\IPMA\FRP\FRP_2000_2024.csv"

# Load CSV into a pandas DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to verify data loading
print(df.head())

df

View csv file on map

In [ ]:
# View map with csv file

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

# Load the dataset
df = pd.read_csv(r"E:\IPMA\FRP\FRP_2000_2024.csv")

# Convert "acq_date" to datetime format and extract year & month
df["acq_date"] = pd.to_datetime(df["acq_date"])
df["year"] = df["acq_date"].dt.year
df["month"] = df["acq_date"].dt.month  # Extract month as integer (1-12)

# Select the year (required) and month (optional)
selected_year = 2020  # Change to desired year (2001-2023)
selected_month = 7  # Set to None to include all months OR set a value (1-12)

# Filter the dataset
if selected_month is not None:
    df = df[(df["year"] == selected_year) & (df["month"] == selected_month)]
else:
    df = df[df["year"] == selected_year]  # Only filter by year

# Convert to GeoDataFrame if data exists
if not df.empty:
    geometry = [Point(xy) for xy in zip(df["longitude"], df["latitude"])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")  # WGS84 CRS

    # Plot the filtered points
    fig, ax = plt.subplots(figsize=(8, 6))
    gdf.plot(ax=ax, color="blue", markersize=10, alpha=0.5)

    title = f"Mapped Locations for {selected_year}"
    if selected_month is not None:
        title += f"-{selected_month:02d}"  # Format as YYYY-MM

    plt.title(title)
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.grid(True)

    plt.show()
else:
    no_data_msg = f"No data available for {selected_year}"
    if selected_month is not None:
        no_data_msg += f"-{selected_month:02d}"
    print(no_data_msg)



View npy file

In [ ]:
import numpy as np

# Load the .npy file
file_path = r"E:\IPMA\SPEI\SPEI3\SPEI3_1979.npy"
data = np.load(file_path)

# Print structure details
print("Shape:", data.shape)
print("Data type:", data.dtype)
print("Preview (first slice):\n", data[0])  # Print first "layer" of the 3D array

In [ ]:
# To view .npy on map

import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Load your data (replace with actual .npy file loading)
file_path = r"E:\IPMA\SPI\SPI3\SPI3_1979.npy"
data = np.load(file_path)

# Check the shape of the data
print(f"Data shape: {data.shape}")  # Should be (time, lat, lon)

# Assume latitudes and longitudes based on the shape of the data
time_size, lat_size, lon_size = data.shape
lat_values = np.linspace(34, 66, lat_size)  # Latitude range from -90 to 90 degrees
lon_values = np.linspace(-12, 36, lon_size)  # Longitude range from -180 to 180 degrees

# Create a 2D meshgrid of latitude and longitude for plotting
lon_grid, lat_grid = np.meshgrid(lon_values, lat_values)

# Plot the data using Cartopy (selecting the first time slice as a sample)
fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_extent([lon_values.min(), lon_values.max(), lat_values.min(), lat_values.max()])

# Add coastlines and gridlines
ax.coastlines()
ax.gridlines(draw_labels=True)

# Plot the first time slice (index 0) from the data
data_slice = data[0, :, :]  # First time slice
c = ax.pcolormesh(lon_grid, lat_grid, data_slice, cmap="viridis", shading='auto', transform=ccrs.PlateCarree())
fig.colorbar(c, ax=ax, orientation="vertical", label="Data Value")

plt.title("Geospatial Data: First Time Slice")
plt.show()

